<center>
    <h1> ILI-286 - Laboratorio #1 </h1>
    <h2> Computación numérica de vectores propios aplicados a PCA </h2>
</center>

| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |


## Tabla de contenidos
1. [Introducción](#intro)
2. [Desarrollo](#Desarrollo)
 1.  [Power Iteration y Rayleigh Quotient](#de1)
 2.  [Naive k-first eigen finder](#de2)
 3.  [Clever k-first eigen finder](#de3)
 4.  [TODO](#de4)
3. [Resultados](#Resultados)
 1.  [Power iteration v/s Rayleigh Quotient iteration](#de5)
 2.  [Naive v/s Clever](#de6)
 3.  [Todos los valores y vectores propios](#de7)
4. [Concluciones](#Concluciones)
5. [Referencias](#Referencias)

<div id='intro' />
## Introducción
Escribir algo aquí.

## Desarrollo

Debemos comenzar cargando las bibliotecas y datos necesarios. Además se definirán algunas variables que se usarán más adelante.

In [ ]:
import numpy as np
from sys import float_info
#Para utilizar %memit
%load_ext memory_profiler

dataset = np.load("arcene.npy")
sigma_x = np.dot((1/(1-dataset.shape[1]))*np.transpose(dataset), dataset)

epsilon = float_info.epsilon

<div id='de1'/>
### 1.- Power Iteration y Rayleigh Quotient

Los siguientes algoritmos requieren una matriz $A_{n\times n}$.
Opcionalmente pueden recibir el vector inicial $x_{n\times 1}$ (por defecto $[1 \dots 1]^{T}$) y un número máximo de iteraciones (por defecto $1000$).

Ambos algoritmos retornan una tupla con el valor y vector propio dominante: $(\lambda, \vec{v})$

In [ ]:
def power_iteration(A, x=None, max_iter=1000):
    #Comprueba las dimensiones de x.
    if x == None: x = np.ones([A.shape[0],1]) #TODO: No sería mejor random??
    elif A.shape[0] != x.shape[0]: raise ValueError("Initial vector error.")
    #Bucle de resolución.
    for j in range(1, max_iter):
        u = x/np.linalg.norm(x)
        x = np.dot(A,u)
        lamb = np.dot(np.transpose(u),x)
    u = x/np.linalg.norm(x)
    return lamb[0,0], u

def rayleigh_quotient_iteration(A, x=None, max_iter=1000):
    #Comprueba las dimensiones de x.
    if x == None: x = np.ones([A.shape[0],1])
    elif A.shape[0] != x.shape[0]: raise ValueError("Initial vector error.")
    #Bucle de resolución.
    I = np.identity(A.shape[0])
    for i in range(1, max_iter):
        u = x/np.linalg.norm(x)
        lamb = np.dot(np.dot(np.transpose(u), A), u)
        C = A - lamb*I
        if np.linalg.det(C) <= epsilon :
            break
        else:
            x = np.linalg.solve(C, u)
    u = x/np.linalg.norm(x)
    return lamb[0,0], u

A = np.array([[3,4,5],[4,2,1],[5,1,8]])
initial = np.array([[0],[1],[0]])
print(power_iteration(A))
#print(rayleigh_quotient_iteration(A))

 * Complejidad algoritmo Power Iteracion:
     $2kn^2 = O(kn^2)$
     
     La complejidad de Power Iteration viene dada principalmente por la cantidad de iteraciones k que alcanze a realizar antes de converger y por el producto punto matriz-vector ubicado dentro del ciclo.
     
     
 * Complejidad algoritmo Rayleigh Quotient Iteration:
     $k(\frac{2}{3} n^3 + 2n^2) = O(kn^3)$
     
     Lo más complejo del algoritmo es el solver. Se asume que utiliza Gaussian Elimination con BS/FS.
 

<div id='de2'/>
### 2.- Naive k-first eigen finder
#### (a) Correctitud:

Explicación dada en la ayudantia #TODO

#### (b) Complejidad:
* Complejidad algoritmo kEigenFinder:
     $2 k_1 k_2 n^2 + 2n = O(k_1 k_2 n^2)$
     
     Donde $k_1$ es el promedio de iteraciones que hará Power Iteration y $k_2$ es la cantidad de iteraciones de kEigenFinder 
 

#### (c) Implementación:

In [ ]:
def kEigenFinder(B, p = None, k = None):
    A = B.copy()
    if p == None: p = np.ones([A.shape[0],1])
    elif A.shape[0] != p.shape[0]: raise ValueError("Initial vector error.") 
    #^Esto no es necesario por que power_iteration hace la misma verificación.
    if k == None: k = A.shape[0] #Def: k=n, mejor k=1?? TODO
    elif k > A.shape[0]: raise ValueError("k is out of range.")
    
    lamb = np.zeros([p.shape[0],1]);
    l = 0;
    v_finales = np.zeros([p.shape[0], p.shape[0]]);
    v = np.zeros([p.shape[0], 1]);
    for j in range(0, k):
        A = A-l*np.dot(v,v.transpose());
        
        l, v = power_iteration(A, p);
        v_finales[:,[j]] = v;
        lamb[j] = l;
    return lamb, v_finales 
A= np.matrix([[3,4,5],[4,2,1],[5,1,8]])
print(kEigenFinder(A))#, np.matrix([1,1,1]).transpose(), 3))

<div id='de3'/>
### 3.- Clever k-first eigen finder
#### (a) Correctitud
#### (b) Complejidad
* Complejidad algoritmo kEigenFinderPP:
     $O(A^p) + k (4n^2 + 4n + p) = O(A^p) + O(kn^2)$ #TODO arreglar O(A^p)
     
     Donde $k$ es la cantidad de iteraciones que realiza kEigenFinderPP
#### (c) Implementación

In [ ]:
def kEigenFinderPP(A, p = 5, k = None):
    if k == None: k = A.shape[0]
    elif k > A.shape[0]: raise ValueError("k is out of range.")
    #Listas donde se guardarán los resulados.
    lamb = [None] * (k + 1)
    v    = [None] * (k + 1)
    x    = [None] * (k + 1)
    #Valores iniciales.
    A_p  = np.linalg.matrix_power(A, p)
    lamb[0] = 0
    v[0] = np.ones([A.shape[0],1])
    x[0] = np.random.random([A.shape[0],1])
    #Bucle de resolución.
    for i in range(0, k):
        A_p -= (lamb[i]**p * np.dot(v[i], np.transpose(v[i])))
        x[i+1] = np.dot(A_p, x[i])
        v[i+1] = x[i+1]/np.linalg.norm(x[i+1])
        lamb[i+1]= np.dot(np.dot(np.transpose(v[i+1]), A), v[i+1])[0][0] #only save the float?
    return lamb[1:], v[1:]

#print(A)
#kEigenFinderPP(A,8)

<div id='de4'/>
### 4.- Modificación a #ToDo.
#### ¿Que debemos modificar?
#### Complejidad computacional
#### Implementación

In [ ]:
def unshifted_qr_algorithm(A):
    Q = np.identity(A.shape[0]);
    R = A;
    Qx = Q
    for j in range (1,9999):
        Q, R = np.linalg.qr(np.dot(R,Q))
        Qx = np.dot(Qx, Q)
    return np.diagonal(np.dot(R, Q))

## TODO: NO FUNCIONA, FALTA ADAPTARLO
def unshifted_qr_algorithm_q(A, q):
    #A = A[0:q,0:q];
    Q = np.identity(A.shape[0]);
    R = A;
    Qx = Q;
    for j in range (1,9999):
        q_r = np.dot(R,Q)
        Q, R = np.linalg.qr(q_r)
        #Q = Q[:,0:q];
        #R = R[0:q,:];
        Qx = np.dot(Qx, Q)
    return [np.diagonal(np.dot(R, Q)), Qx]
A = np.matrix([[3,4,5],
               [4,2,1],
               [5,1,8]]);
print(unshifted_qr_algorithm(A))
print(unshifted_qr_algorithm_q(A,1))
print(unshifted_qr_algorithm_q(A,2))
print(unshifted_qr_algorithm_q(A,3))

## Resultados
<div id='de5'/>
### 5.- Power iteration v/s Rayleigh Quotient iteration.

In [ ]:
#Comentado por carga
#%timeit power_iteration(sigma_x, max_iter=20)
#%memit  power_iteration(sigma_x, max_iter=20)

#%timeit rayleigh_quotient_iteration(sigma_x, max_iter=20)
#%memit  rayleigh_quotient_iteration(sigma_x, max_iter=20)

#TODO: comparacion y concluciones...
<div id='de6'/>
### 6.- Naive v/s Clever v/s

In [ ]:
K = [10, 100, 1000]
P = 10
k = 1
#%timeit kEigenFinder(sigma_x, None, k)
#%memit  kEigenFinder(sigma_x, None, k)

#%timeit kEigenFinder(sigma_x, P, k)
#%memit  kEigenFinder(sigma_x, P, k)

<div id='de7'/>
### 6.- Todos los valores y vectores propios

## Concluciones

## Referencias